# Classifier Training

## Common Imports and Settings

In [2]:
import sys

sys.path.insert(0, "/home/jupyter/ML_for_slideseq/src")

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import torchvision
import torch
import pytorch_lightning as pl
import tissue_purifier as tp

matplotlib.rcParams["figure.dpi"] = 200

In [5]:
data_folder = "/home/jupyter/data/slide-seq/original_data/"
#data_folder = "../slide-seq-data"

df_wt1 = pd.read_csv(os.path.join(data_folder, "wt_1.csv"))
df_wt2 = pd.read_csv(os.path.join(data_folder, "wt_2.csv"))
df_wt3 = pd.read_csv(os.path.join(data_folder, "wt_3.csv"))
df_dis1 = pd.read_csv(os.path.join(data_folder, "sick_1.csv"))
df_dis2 = pd.read_csv(os.path.join(data_folder, "sick_2.csv"))
df_dis3 = pd.read_csv(os.path.join(data_folder, "sick_3.csv"))

In [6]:
NUM_WORKERS = 8
BATCH_SIZE = 128
SEED = 1
MAX_EPOCHS = 100
INPUT_SIZE = 224
PIXEL_SIZE = 4.0
CROP_SIZE = INPUT_SIZE
INPUT_CHANNELS = 9
N_ELEMENT_MIN = 200
N_CROPS_TEST = 150
# Model parameters:
NUM_FTRS = 128
PROJECTION_OUT_DIM = 128
# Optimizer parameters:
LEARNING_RATE = 0.001
IS_SCHEDULED = True
SCHEDULER_STEP = 3
SCHEDULER_GAMMA = 0.995

## Define a training pipe helper function

Here we use helper functions

In [38]:
import lightly
import pytorch_lightning as pl
from pytorch_lightning.loggers import NeptuneLogger

def get_embedding_dataloader(encoder, dataloader, epochs=1, batch_size=500):
    embeddings, labels, _ = encoder.embed(dataloader, epochs)
    embedding_dataset = torch.utils.data.TensorDataset(embeddings, labels)
    return torch.utils.data.DataLoader(embedding_dataset, batch_size=batch_size)


def define_and_train_classifier(test_dfs, test_labels, train_dfs, train_labels, save_name, loss=None, epochs=100, data_multiplicative_factor=100):
    model = tp.model_utils.helpers.define_model(
        number_of_channels=INPUT_CHANNELS,
        num_of_filters=NUM_FTRS,
        projection_out_dim=PROJECTION_OUT_DIM
    )
    ckpt = torch.load("model.pth")
    model.backbone.load_state_dict(ckpt["resnet18_parameters"])
    model = model.to("cuda")
    encoder = tp.model_utils.TrainedEmbedding(model)
    
    print("Test Data")
    test_sparse_images = tp.data_utils.SparseImage.from_pandas(
        test_dfs, x="x", y="y", category="cell_type", pixel_size=PIXEL_SIZE, padding=10
    )
    print("Train Data:")
    train_sparse_images = tp.data_utils.SparseImage.from_pandas(
        train_dfs, x="x", y="y", category="cell_type", pixel_size=PIXEL_SIZE, padding=10
    )
    # Image dataloaders
    test_dataloader = tp.data_utils.helpers.define_dataloader_test(
        sparse_images=test_sparse_images,
        labels_sparse_images=test_labels,
        input_size=INPUT_SIZE,
        crop_size=CROP_SIZE,
        n_crops_test=N_CROPS_TEST,
        n_element_min=1000,
        batch_size=BATCH_SIZE,
    )
    train_dataloader = tp.data_utils.helpers.define_dataloader_train_classifier(
        sparse_images=train_sparse_images,
        labels_sparse_images=train_labels,
        batch_size=BATCH_SIZE,
        crop_size=CROP_SIZE,
        input_size=INPUT_SIZE,
        simclr_output=False,
        n_element_min=1000
    )
    # Embedding Dataloaders based on embedded images
    test_embedding_dataloader = get_embedding_dataloader(encoder, test_dataloader)
    train_embedding_dataloader = get_embedding_dataloader(encoder, train_dataloader, epochs=data_multiplicative_factor)
    
    # Just not to store unnecessary data on GPU
    del train_dataloader 
    del test_dataloader
    
    classifier = tp.model_utils.EmbeddingClassifier(
        embedding_out_dim=NUM_FTRS, loss=loss, out_dim=2, max_epochs=MAX_EPOCHS
    )
    
    ## Uncomment this if you want to use neptune logger ###
    # neptune_logger = NeptuneLogger(
    #     api_key="key",
    #     project_name='fedorgrab/slide-seq-contrastive', # change this to your project
    #     experiment_name="Linear Classifier of Embeddings",
    #     tags=["Embedding Classification"]
    # )
    ###
    gpus = 1
    trainer = pl.Trainer(max_epochs=epochs, gpus=gpus, progress_bar_refresh_rate=0, log_every_n_steps=1)#, logger=neptune_logger)
    trainer.fit(classifier, train_embedding_dataloader, test_embedding_dataloader)
    torch.save(classifier.fc.state_dict(), save_name)
#     neptune_logger.experiment.log_artifact(save_name)
    return classifier

## Train Classifier 1: Test sample – 1; train samples – 2, 3,

In [39]:
classifier_1 = define_and_train_classifier(
    test_dfs=[df_wt1, df_dis1],
    test_labels=[0, 1],
    train_dfs=[df_wt2, df_wt3, df_dis2, df_dis3],
    train_labels=[0, 0, 1, 1],
    save_name="classifier_1_fc.pt",
    data_multiplicative_factor=10
)

Test Data
number of elements ---> 31659
The dense shape of the image is -> torch.Size([9, 1168, 1168])
number of elements ---> 27194
The dense shape of the image is -> torch.Size([9, 1166, 1170])
Train Data:
number of elements ---> 33059
The dense shape of the image is -> torch.Size([9, 1170, 845])
number of elements ---> 39206
The dense shape of the image is -> torch.Size([9, 1169, 1170])
number of elements ---> 42776
The dense shape of the image is -> torch.Size([9, 1170, 1170])
number of elements ---> 33441
The dense shape of the image is -> torch.Size([9, 1154, 1155])


NeptuneLogger will work in online mode
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


https://app.neptune.ai/fedorgrab/slide-seq-contrastive/e/SDCL-126



  | Name | Type             | Params
------------------------------------------
0 | loss | CrossEntropyLoss | 0     
1 | fc   | Linear           | 258   
------------------------------------------
258       Trainable params
0         Non-trainable params
258       Total params
0.001     Total estimated model params size (MB)
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this

## Train Classifier 2: test sample – 2; train samples – 1, 3,

In [8]:
classifier_2 = define_and_train_classifier(
    test_dfs=[df_wt2, df_dis2],
    test_labels=[0, 1],
    train_dfs=[df_wt1, df_wt3, df_dis1, df_dis3],
    train_labels=[0, 0, 1, 1]
)
torch.save(classifier_2.fc.state_dict(), "classifier_2_fc.pt")
neptune.experiment.log_artifact("classifier_2_fc.pt")

Test Data:
number of elements ---> 33059
The dense shape of the image is -> torch.Size([9, 1170, 845])
number of elements ---> 42776
The dense shape of the image is -> torch.Size([9, 1170, 1170])
Train Data:
number of elements ---> 31659
The dense shape of the image is -> torch.Size([9, 1168, 1168])
number of elements ---> 39206
The dense shape of the image is -> torch.Size([9, 1169, 1170])
number of elements ---> 27194
The dense shape of the image is -> torch.Size([9, 1166, 1170])
number of elements ---> 33441
The dense shape of the image is -> torch.Size([9, 1154, 1155])


NeptuneLogger will work in online mode
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


https://app.neptune.ai/fedorgrab/slide-seq-contrastive/e/SDCL-105



  | Name          | Type     | Params
-------------------------------------------
0 | resnet_simclr | SimCLR   | 11.3 M
1 | fc            | Linear   | 258   
2 | accuracy      | Accuracy | 0     
-------------------------------------------
258       Trainable params
11.3 M    Non-trainable params
11.3 M    Total params
45.177    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

NameError: name 'neptune' is not defined